In [1]:
#import libraries
import requests
import re
from bs4 import BeautifulSoup
from requests import get
url = "https://hackr.io/tutorials/learn-typography"
response = get(url)
#We’ll now create a BeautifulSoup object, or a parse tree. This object takes as its arguments the page.text document from Requests (the content of the server’s response) and then parses it from Python’s built-in html.parser.
soup = BeautifulSoup(response.text, 'html.parser')

# Pull all text from the tuturial divison that contains course listing
courses = soup.find_all('div', class_ ="tut-list tut-list-new tut-row")
print(type(courses))
print(len(courses))

<class 'bs4.element.ResultSet'>
4


Now we’ll select each container, and extract, by turn, each item of interest:

- Course title
- Votes
- Author
- Links (Course, Author, Hackr)
- Course Features: (Free / Paid) (video / book ) (Beginner / Intermediate / Advanced)


In [2]:
# Lists to store the scraped data in
CourseTitles = []
Votes = []
CourseLink = []
AuthorName = []
AuthorLink = []
HackrLink = []
CourseCost = []
CourseType = []
StudentType = []


    
for course in courses:
    CourseTitle = course.div.find('span', class_ = 'tutorial-title-txt').text.strip()
    CourseTitles.append(CourseTitle)


    NoOfVote  = course.div.find('span', class_ = 'count').text.strip()
    Votes.append(NoOfVote)

    Link1 = course.find("div", class_="title-links no-margin").find('a')["href"]
    CourseLink.append(Link1)
    
    Name = course.find("a", class_="author no-marginright paddingleft-xs")
    if Name is None:
        AuthorName.append(None)
    else:
        Names=Name.text.strip()
        AuthorName.append(Names)
    
    auth  = course.find("div", class_="action author").find('a')
    if auth is None:
        AuthorLink.append(None)
    else:
        Link2 = auth["href"]
        AuthorLink.append(Link2)
    
    hackr = course.find("a", attrs={'href': re.compile("https://hackr.io/tutorial/")}) 
    if hackr is None:
        HackrLink.append(None)
    else:
        Link3=hackr.get("href")
        HackrLink.append(Link3)
        
    features  = course.find_all('div', class_ = "tut-label-group marginright-sm")
    for future in features:
        x=future.text.split()
    feat1 = x[0]
    if len(x) > 2:
        feat2 = x[1]
        feat3 = x[2]
    elif x[1] == ("Beginner" or "Advanced"):
        feat2 =None
        feat3= x[1]
    else:
        feat2 = x[1]
        feat3 = None
    CourseCost.append(feat1)
    CourseType.append(feat2)
    StudentType.append(feat3)
        

print (f"Courses : {CourseTitles}\n")
print (f"Votes : {Votes}\n")
print (f"CourseLink : {CourseLink}\n")
print (f"Author : {AuthorName}\n")
print (f"AuthorLink : {AuthorLink}\n")
print (f"HackrLink : {HackrLink}\n")
print (f"CourseLink : {CourseLink}\n")
print (f"CourseCost : {CourseCost}\n")
print (f"CourseType : {CourseType}\n")
print (f"StudentType : {StudentType}\n")


Courses : ['Better Web Typography in 13 Simple Steps', 'Power of the Pen – A Hand Lettering Tutorial', 'Typography Course by Design Lab', 'Typography Essentials ? A Getting Started Guide']

Votes : ['4', '2', '2', '+1']

CourseLink : ['http://www.creativebloq.com/typography/better-web-typography-few-simple-steps-5132803', 'https://www.toptal.com/designers/typography/hand-lettering-tutorial', 'http://trydesignlab.com/typography-course/', 'http://freelancefolder.com/typography-essentials-a-getting-started-guide/']

Author : [None, 'Steph', 'Jack', None]

AuthorLink : [None, 'https://hackr.io/steph-smith', 'https://hackr.io/jack-avery-1', None]

HackrLink : [None, 'https://hackr.io/tutorial/power-of-the-pen-a-hand-lettering-tutorial', 'https://hackr.io/tutorial/typography-course-by-try-design-lab', None]

CourseLink : ['http://www.creativebloq.com/typography/better-web-typography-few-simple-steps-5132803', 'https://www.toptal.com/designers/typography/hand-lettering-tutorial', 'http://tryd

In [3]:
import pandas as pd
TypographyCourses = pd.DataFrame({'Course': CourseTitles,
'Votes': Votes,
"Course Link":CourseLink,
"Author ":AuthorName,
"Author Link":AuthorLink,
"Hackr Link":HackrLink,
"Course Cost":CourseCost,
"Course Type":CourseType,
"Course Cost":CourseCost,
"Student Type":StudentType
})
print(TypographyCourses.info())
TypographyCourses

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Course        4 non-null      object
 1   Votes         4 non-null      object
 2   Course Link   4 non-null      object
 3   Author        2 non-null      object
 4   Author Link   2 non-null      object
 5   Hackr Link    2 non-null      object
 6   Course Cost   4 non-null      object
 7   Course Type   3 non-null      object
 8   Student Type  4 non-null      object
dtypes: object(9)
memory usage: 416.0+ bytes
None


,Course,Votes,Course Link,Author,Author Link,Hackr Link,Course Cost,Course Type,Student Type
0,Better Web Typography in 13 Simple Steps,4,http://www.creativebloq.com/typography/better-...,None,None,None,Free,Video,Beginner
1,Power of the Pen – A Hand Lettering Tutorial,2,https://www.toptal.com/designers/typography/ha...,Steph,https://hackr.io/steph-smith,https://hackr.io/tutorial/power-of-the-pen-a-h...,Free,None,Beginner
2,Typography Course by Design Lab,2,http://trydesignlab.com/typography-course/,Jack,https://hackr.io/jack-avery-1,https://hackr.io/tutorial/typography-course-by...,Paid,Video,Beginner
3,Typography Essentials ? A Getting Started Guide,+1,http://freelancefolder.com/typography-essentia...,None,None,None,Free,Video,Beginner


In [4]:
TypographyCourses.to_csv('TypographyCourses.csv')